In [ ]:
import pandas as pd
import numpy as np
import re
from expense_data.models import Expense_Data
from expense_category.models import Expense_Category
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.svm import LinearSVC 
from sklearn.pipeline import Pipeline 
from sklearn.feature_selection import SelectKBest, chi2 

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LinearRegression

In [ ]:
expense_Data = pd.DataFrame(list(Expense_Data.objects.all().values())) 
    
# Printing the dataswet shape 
print ("Dataset Length: ", len(expense_Data)) 
print ("Dataset Shape: ", expense_Data.shape) 

# Printing the dataset obseravtions 
print ("Dataset: ",expense_Data.head())

In [ ]:
nltk.download('stopwords')
stemmer= SnowballStemmer('english')
words = stopwords.words('english')
expense_Data['cleaned'] = expense_Data['Description'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ",x).replace('Debit Card Purchase','').split() if i not in words]).lower())


In [ ]:
is_NotNoneType =np.logical_not(np.logical_or(expense_Data['ExpenseType_id']==2,expense_Data['ExpenseType_id'] == 1))
new_data = expense_Data[is_NotNoneType]
# Seperating the target variable 
X = new_data['cleaned']
Y = new_data['ExpenseType_id'] 

# Spliting the dataset into train and test 2
X_train, X_test, y_train, y_test = train_test_split(  
X, Y, test_size = 0.3, random_state = 1) 

In [ ]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), stop_words = 'english', sublinear_tf = True)),
                    ('chi', SelectKBest(chi2, k='all')),
                    ('clf', LinearSVC(C=1.0, penalty='l1',max_iter=300,dual=False))])
# Creating the classifier object 
model = pipeline.fit(X_train, y_train)

In [ ]:
y_pred = clf_object.predict(X_test) 

In [ ]:
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred)) 

print ("Accuracy : ", accuracy_score(y_test,y_pred)*100) 

print("Report : ", classification_report(y_test, y_pred)) 

In [ ]:
is_NoneType =np.logical_or(expense_Data['ExpenseType_id']==2,expense_Data['ExpenseType_id'] == 1)
data_NoneType =  expense_Data[is_NoneType]
y_pred_New = model.predict(data_NoneType['cleaned']) 
data['ExpenseType_id'] = y_pred_New